# GIS Image Stack Pre-Processing 

This notebook implements a processing pipeline to transform globally available data types into spatially aligned image stacks with uniform pixel sizes which involve the following steps. The image stack is represented as individual GeoTiff files for each data type that are cropped to an Area of Interest (AOI), specifically a UTM zone, and resampled at teh same pixel resolution. The filesystem structure is organized as described below.

This notebook produces an output folder for each data type of the form `PK_42N` and withing that folder are four GeoTiff files that represent the four processing stages implemented in this notebook (number 1 to 4). The forth file is the final output GeoTiff which represents the cropped and resampled raster data that constitutes a single layer in the image stack. 

## Expected File System Structure (for PK Zone 42N)

There is a folder for each data type at the same level as this notebook <code>prep_geospatial_data.ipynb</code>. With the exception of the Rainfall data which has a folder structure for the specified country and UTM zone, the other data types contain a global GeoTiff file directly within the data type folder that is used as input for this notebook. For all data types, output folders are created that indicate the two-letter country code followed by the UTM zone (e.g., PK_42N). The output from this notebook are cropped and resampled GeoTiff files that constitute a single layer in the image stack. These files will be used to create AOI image tiles using a separate notebook <code>prep_aoi_image_tiles.ipynb</code>

<!-- <code><span style="color: blue;">Flood/</span>
&nbsp;&nbsp;&nbsp;&nbsp;<span style="color: green;">landscan-global-2022.tif</span>
&nbsp;&nbsp;&nbsp;&nbsp;<span style="color: red;">output/PK_42N</span>
<br>
<span style="color: blue;">Landcover/</span>
&nbsp;&nbsp;&nbsp;&nbsp;<span style="color: green;">esalc_2020.tif</span>
&nbsp;&nbsp;&nbsp;&nbsp;<span style="color: red;">output/PK_42N</span>
<br> -->
<code><span style="color: blue;">Nightlights/</span>
&nbsp;&nbsp;&nbsp;&nbsp;<span style="color: green;">VNL_v22_npp-j01_2022_global_vcmslcfg_c202303062300_median.tif</span>
&nbsp;&nbsp;&nbsp;&nbsp;<span style="color: red;">output/PK_42N/</span>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style="color: blue;">VNL_v22_npp-j01_2022_global_vcmslcfg_median_PK_42N_4_resampled_average.tif</span>
<br>
<span style="color: blue;">Population/</span>
&nbsp;&nbsp;&nbsp;&nbsp;<span style="color: green;">landscan-global-2022.tif</span>
&nbsp;&nbsp;&nbsp;&nbsp;<span style="color: red;">output/PK_42N/</span>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style="color: blue;">landscan-global-2022_PK_42N_4_resampled_average.tif</span>
<br>
<span style="color: blue;">Rainfall/</span>
&nbsp;&nbsp;&nbsp;&nbsp;<span style="color: green;">Chirps_2023/PK_42N/output/chirps-v2.0.2023_PK_42N_avg.tif</span>
&nbsp;&nbsp;&nbsp;&nbsp;<span style="color: red;">output/PK_42N</span>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style="color: blue;">chirps-v2.0.2023_PK_42N_avg_PK_42N_4_resampled_average.tif</span></code>



<!-- - **Crop Data to UTM Zone**: Crop the (geographic) source data to the appropriate UTM zone that encompasses the region of interest (adding a +/- 1 deg buffer in longitude). Adding a buffer is necessary to enable image tiles near the border to be created.
- **Standardize NoData / NaN Values**: Replace NoData values and NaNs with a fixed value to ensures consistency across data types and to facilitate pixel imputation. 
- **Pixel Imputation (for NoData)**: This step is required to infill pixels that do not contain valid data, but requires further investigation to determine the appropriate sequence in the processing pipeline and the algorithm for pixel value imputation.
- **Transform the Geographic Data to UTM**: Transform the geographic data to UTM using the appropriate UTM zone.
- **Resample Data** : Resample each data type to a consistent pixel size. -->


## UTM Zones for Reference

```
EPSG:32628  28N	 18°W to 12°W
EPSG:32629  29N	 12°W to 6°W
EPSG:32630  30N	 6°W  to 0°
EPSG:32631  31N	 0°   to 6°E
EPSG:32632  32N  6°E  to 12°E
EPSG:32633  33N  12°E to 18°E
EPSG:32634  34N	 18°E to 24°E
EPSG:32635  35N	 24°E to 30°E
EPSG:32636  36N	 30°E to 36°E
EPSG:32637  37N	 36°E to 42°E
EPSG:32638  38N	 42°E to 48°E
EPSG:32639  39N	 48°E to 54°E
EPSG:32640  40N	 54°E to 60°E
EPSG:32641  41N	 60°E to 66°E
EPSG:32642  42N	 66°E to 72°E
EPSG:32643  43N	 72°E to 78°E
EPSG:32644  44N	 78°E to 84°E
EPSG:32645  45N	 84°E to 90°E
```

In [1]:
import os
from dataclasses import dataclass

# Import module that contains several convenience functions (e.g., gdal wrappers)
from gist_utils import *

# Adding path to gdal commands for local system
os.environ['PATH'] += ':/Users/billk/miniforge3/envs/py39-pt/bin/' 

## 1 Set Country Code and UTM Zone

The only input settings required in this notebook are the two-letter country code, UTM zone, and the latitude band for the specified UTM zone.

In [2]:
country_code  = 'PK'
utm_zone      = '42N'
utm_code = get_epsg_code(utm_zone)

case = country_code + '_' + utm_zone 

## 2 Configure Data Type

This notebook is intended to be executed for a single data type at a time. Each of the data classes below can be pre-configured, and then in the following section, a single data type can be specified for execution which will produce intermediate output (*.tif ) files leading up to the final processed GeoTiff. 

In [3]:
@dataclass(frozen=True)
class DatasetConfig_Flood:                  
    INPUT_TIF:    str  = "./dfo_3696_from_20100727_to_20101115_band5.tif"     
    OUT_DIR:      str  = "./Flood/output/{country_code}_{utm_zone}/"    # Output folder to store processed data     
    NODATA_SRC:   int  = -999                                           # NoData value in src data  
    NODATA_SET:   int  = -999                                           # NoData value to fill in processed data     
    RESAMPLE_ALG: str  = 'mode'
    GDAL_INFO:    bool = True
    
    def get_input_tif(self):
        return self.INPUT_TIF.format(country_code=self.COUNTRY_CODE, utm_zone=self.UTM_ZONE)
    
    def get_out_dir(self):
        return self.OUT_DIR.format(country_code=self.COUNTRY_CODE, utm_zone=self.UTM_ZONE)

In [4]:
@dataclass(frozen=True)
class DatasetConfig_Landcover:                  
    INPUT_TIF:    str  = "./esalc_2020.tif"      
    OUT_DIR:      str  = "./Landcover/output/{country_code}_{utm_zone}/"   # Output folder to store processed data
    NODATA_SRC:   int = -999                                               # NoData value in src data  
    NODATA_SRC:   int  = -999                                              # NoData value to fill in processed data     
    NODATA_SET:   int  = -999  
    RESAMPLE_ALG: str  = 'mode'     
    GDAL_INFO:    bool = False
        
    def get_input_tif(self):
        return self.INPUT_TIF.format(country_code=self.COUNTRY_CODE, utm_zone=self.UTM_ZONE)
    
    def get_out_dir(self):
        return self.OUT_DIR.format(country_code=self.COUNTRY_CODE, utm_zone=self.UTM_ZONE)

In [5]:
@dataclass(frozen=True)
class DatasetConfig_Nightlights:
    COUNTRY_CODE: str  
    UTM_ZONE:     str
    INPUT_TIF:    str = "./Nightlights/VNL_v22_npp-j01_2022_global_vcmslcfg_median.tif"  # Source data GeoTiff 
    OUT_DIR:      str = "./Nightlights/output/{country_code}_{utm_zone}/"                # Output folder to store processed data
    NODATA_SRC:   int = -999                                                             # NoData value in src data  
    NODATA_SRC:   int = -999                                                             # NoData value to fill in processed data
    NODATA_SET:   int = -999                                                             
    RESAMPLE_ALG: str = 'average'
    GDAL_INFO:   bool = False
        
    def get_input_tif(self):
        return self.INPUT_TIF.format(country_code=self.COUNTRY_CODE, utm_zone=self.UTM_ZONE)
    
    def get_out_dir(self):
        return self.OUT_DIR.format(country_code=self.COUNTRY_CODE, utm_zone=self.UTM_ZONE)

In [6]:
@dataclass(frozen=True)
class DatasetConfig_Population:
    COUNTRY_CODE: str  
    UTM_ZONE:     str
    INPUT_TIF:    str = "./Population/landscan-global-2022.tif"           # Source data GeoTiff
    OUT_DIR:      str = "./Population/output/{country_code}_{utm_zone}/"  # Output folder to store processed data
    NODATA_SRC:   int = -2147483647                                       # NoData value in src data  
    NODATA_SET:   int = -999                                              # NoData value to fill in processed data
    RESAMPLE_ALG: str = 'average'
    GDAL_INFO:    bool = False
        
    def get_input_tif(self):
        return self.INPUT_TIF.format(country_code=self.COUNTRY_CODE, utm_zone=self.UTM_ZONE)
    
    def get_out_dir(self):
        return self.OUT_DIR.format(country_code=self.COUNTRY_CODE, utm_zone=self.UTM_ZONE)

In [7]:
@dataclass(frozen=True)
class DatasetConfig_Rainfall:
    COUNTRY_CODE: str  
    UTM_ZONE:     str                     
    INPUT_TIF:    str   = "./Rainfall/Chirps_2023/{country_code}_{utm_zone}/output/chirps-v2.0.2023_{country_code}_{utm_zone}_avg.tif"     
    OUT_DIR:      str   = "./Rainfall/output/{country_code}_{utm_zone}/"           
    NODATA_SRC:   float = 3.4028234663852886e+38                   
    NODATA_SET:   int   = -999 
    RESAMPLE_ALG: str   = 'average'
    GDAL_INFO:    bool  = True
        
    def get_input_tif(self):
        return self.INPUT_TIF.format(country_code=self.COUNTRY_CODE, utm_zone=self.UTM_ZONE)
    
    def get_out_dir(self):
        return self.OUT_DIR.format(country_code=self.COUNTRY_CODE, utm_zone=self.UTM_ZONE)

## 3 Configure AOI

The AOI data class below is used to configure the AOI for all the data types which serves to specify the cropped region for the image stack along with the re-sampling size in meters. The UTM zone and latitude boundaries are determined visually by inspecting the rgeion of interest (e.g, country) and the associated DHS survery data locations.

In [8]:
@dataclass(frozen=True)
class AOIConfig:
    UTM_CODE:    str                         
    RESAMPLE:    float = 400                       # Resample data in UTM x & y (meters)
    UTM_BUF_DEG:   int = 1.0                       # Extended buffer in deg longitude beyond UTM zone
    LAT_NORTH:   float = 38.0                      # Define max latitude for AOI
    LAT_SOUTH:   float = 23.0                      # Define min latitude for AOI

#-------------------------------------------------------------------
# Configure data type here (uncomment one line from the list below)
#-------------------------------------------------------------------
#data_config = DatasetConfig_Flood(COUNTRY_CODE=country_code, UTM_ZONE=utm_zone)
#data_config = DatasetConfig_Landcover(COUNTRY_CODE=country_code, UTM_ZONE=utm_zone)
#data_config = DatasetConfig_Nightlights(COUNTRY_CODE=country_code, UTM_ZONE=utm_zone)

#data_config = DatasetConfig_Population(COUNTRY_CODE=country_code, UTM_ZONE=utm_zone)
#data_config = DatasetConfig_Nightlights(COUNTRY_CODE=country_code, UTM_ZONE=utm_zone)
data_config = DatasetConfig_Rainfall(COUNTRY_CODE=country_code, UTM_ZONE=utm_zone)

aoi_config  = AOIConfig(UTM_CODE=utm_code)

if not os.path.exists(data_config.get_out_dir()):
    os.makedirs(data_config.get_out_dir())

##  Information Summary: Source Data

In [9]:
if data_config.GDAL_INFO:
    run_gdalinfo(data_config.get_input_tif())

Driver: GTiff/GeoTIFF
Files: ./Rainfall/Chirps_2023/PK_42N/output/chirps-v2.0.2023_PK_42N_avg.tif
Size is 160, 300
Coordinate System is:
GEOGCRS["WGS 84",
    ENSEMBLE["World Geodetic System 1984 ensemble",
        MEMBER["World Geodetic System 1984 (Transit)"],
        MEMBER["World Geodetic System 1984 (G730)"],
        MEMBER["World Geodetic System 1984 (G873)"],
        MEMBER["World Geodetic System 1984 (G1150)"],
        MEMBER["World Geodetic System 1984 (G1674)"],
        MEMBER["World Geodetic System 1984 (G1762)"],
        MEMBER["World Geodetic System 1984 (G2139)"],
        ELLIPSOID["WGS 84",6378137,298.257223563,
            LENGTHUNIT["metre",1]],
        ENSEMBLEACCURACY[2.0]],
    PRIMEM["Greenwich",0,
        ANGLEUNIT["degree",0.0174532925199433]],
    CS[ellipsoidal,2],
        AXIS["geodetic latitude (Lat)",north,
            ORDER[1],
            ANGLEUNIT["degree",0.0174532925199433]],
        AXIS["geodetic longitude (Lon)",east,
            ORDER[2],
          

##  4 Crop Data to AOI (UTM Zone) 
Crop the source data to the appropriate UTM zone with a buffer in longitude. Adding a buffer is necessary to enable image tiles near the border to be created.

In [10]:
# Define the longitude booundaries for the specified UTM zone
utm_west_lon, utm_east_lon = utm_zone_longitude_bounds(aoi_config.UTM_CODE)

# Define AOI to encompass local UTM zone (+/- small buffer). Choose latitude to cover data for region
ul_lat, ul_lon = aoi_config.LAT_NORTH, utm_west_lon - aoi_config.UTM_BUF_DEG
lr_lat, lr_lon = aoi_config.LAT_SOUTH, utm_east_lon + aoi_config.UTM_BUF_DEG

# Print the results
print("\n")
print(f"Upper Left Lat: {ul_lat}")
print(f"Upper Left Lon: {ul_lon}")
print(f"Lower Right Lat: {lr_lat}")
print(f"Lower Right Lon: {lr_lon}")

epsg_code:  EPSG:32642


Upper Left Lat: 38.0
Upper Left Lon: 65.0
Lower Right Lat: 23.0
Lower Right Lon: 73.0


In [11]:
# First, extract just the filename from the input TIFF path, removing the directory structure.
input_tif = data_config.get_input_tif()
input_tif_filename = os.path.basename(data_config.get_input_tif())

step = "_" + case + "_1_crop_geo.tif"

intermediate_tif = os.path.join(data_config.get_out_dir(), os.path.splitext(input_tif_filename)[0] + step)

print("input_tif:          ", input_tif)
print("\n")
print("intermediate_tif:   ", intermediate_tif)
print("\n")

gdal_crop(input_tif, intermediate_tif, ul_lon, ul_lat, lr_lon, lr_lat, False)

input_tif:           ./Rainfall/Chirps_2023/PK_42N/output/chirps-v2.0.2023_PK_42N_avg.tif


intermediate_tif:    ./Rainfall/output/PK_42N/chirps-v2.0.2023_PK_42N_avg_PK_42N_1_crop_geo.tif


Input file size is 160, 300
0...10...20...30...40...50...60...70...80...90...100 - done.



##  5 Set NoData Value

In [12]:
input_tif = intermediate_tif 

step = "_" + case + "_2_nodata.tif"

intermediate_tif = os.path.join(data_config.get_out_dir(), os.path.splitext(input_tif_filename)[0] + step)

print("input_tif:        ", input_tif)
print("\n")
print("intermediate_tif: ", intermediate_tif)
print("\n")

# Replace nodata values
gdal_replace_nodata(input_tif, intermediate_tif, data_config.NODATA_SET, data_config.NODATA_SRC, False)

input_tif:         ./Rainfall/output/PK_42N/chirps-v2.0.2023_PK_42N_avg_PK_42N_1_crop_geo.tif


intermediate_tif:  ./Rainfall/output/PK_42N/chirps-v2.0.2023_PK_42N_avg_PK_42N_2_nodata.tif


Creating output file that is 160P x 300L.
Processing ./Rainfall/output/PK_42N/chirps-v2.0.2023_PK_42N_avg_PK_42N_1_crop_geo.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.



## Information Summary: Cropped Data (with NoData set)

In [13]:
if data_config.GDAL_INFO:
    run_gdalinfo(intermediate_tif)

Driver: GTiff/GeoTIFF
Files: ./Rainfall/output/PK_42N/chirps-v2.0.2023_PK_42N_avg_PK_42N_2_nodata.tif
Size is 160, 300
Coordinate System is:
GEOGCRS["WGS 84",
    ENSEMBLE["World Geodetic System 1984 ensemble",
        MEMBER["World Geodetic System 1984 (Transit)"],
        MEMBER["World Geodetic System 1984 (G730)"],
        MEMBER["World Geodetic System 1984 (G873)"],
        MEMBER["World Geodetic System 1984 (G1150)"],
        MEMBER["World Geodetic System 1984 (G1674)"],
        MEMBER["World Geodetic System 1984 (G1762)"],
        MEMBER["World Geodetic System 1984 (G2139)"],
        ELLIPSOID["WGS 84",6378137,298.257223563,
            LENGTHUNIT["metre",1]],
        ENSEMBLEACCURACY[2.0]],
    PRIMEM["Greenwich",0,
        ANGLEUNIT["degree",0.0174532925199433]],
    CS[ellipsoidal,2],
        AXIS["geodetic latitude (Lat)",north,
            ORDER[1],
            ANGLEUNIT["degree",0.0174532925199433]],
        AXIS["geodetic longitude (Lon)",east,
            ORDER[2],
      

In [14]:
# Open the GeoTIFF file
with rasterio.open(intermediate_tif) as dataset:

    # Access bounding box (bounds)
    bounds = dataset.bounds
    print(f"Bounds:    BoundingBox(left={bounds.left:.3f}, bottom={bounds.bottom:.3f}, right={bounds.right:.3f}, top={bounds.top:.3f})")

    # Access the CRS directly
    crs = dataset.crs

    # Print the EPSG code, if available
    if dataset.crs.is_epsg_code:
        print("EPSG Code:", crs.to_epsg())
    else:
        print("CRS is not an EPSG: ", dataset.crs.to_string())

Bounds:    BoundingBox(left=65.000, bottom=23.000, right=73.000, top=38.000)
EPSG Code: 4326


## 6 Project Geographic Data to UTM

Project the geographic data to UTM so that image tiles consist of uniform pixel sizes as measured by their edge extents on the ground.

In [15]:
input_tif = intermediate_tif 

step = "_" + case + "_3_utm.tif"

intermediate_tif = os.path.join(data_config.get_out_dir(), os.path.splitext(input_tif_filename)[0] + step)

print("input_tif:        ", input_tif)
print("\n")
print("intermediate_tif: ", intermediate_tif)
print("\n")

transform_to_utm(input_tif, intermediate_tif, utm_code, False)

input_tif:         ./Rainfall/output/PK_42N/chirps-v2.0.2023_PK_42N_avg_PK_42N_2_nodata.tif


intermediate_tif:  ./Rainfall/output/PK_42N/chirps-v2.0.2023_PK_42N_avg_PK_42N_3_utm.tif


Creating output file that is 152P x 310L.
Processing ./Rainfall/output/PK_42N/chirps-v2.0.2023_PK_42N_avg_PK_42N_2_nodata.tif [1/1] : 0Using internal nodata values (e.g. -999) for image ./Rainfall/output/PK_42N/chirps-v2.0.2023_PK_42N_avg_PK_42N_2_nodata.tif.
Copying nodata values from source ./Rainfall/output/PK_42N/chirps-v2.0.2023_PK_42N_avg_PK_42N_2_nodata.tif to destination ./Rainfall/output/PK_42N/chirps-v2.0.2023_PK_42N_avg_PK_42N_3_utm.tif.
...10...20...30...40...50...60...70...80...90...100 - done.



##  Information Summary: UTM Data

In [16]:
if data_config.GDAL_INFO:
    run_gdalinfo(intermediate_tif)

Driver: GTiff/GeoTIFF
Files: ./Rainfall/output/PK_42N/chirps-v2.0.2023_PK_42N_avg_PK_42N_3_utm.tif
Size is 152, 310
Coordinate System is:
PROJCRS["WGS 84 / UTM zone 42N",
    BASEGEOGCRS["WGS 84",
        DATUM["World Geodetic System 1984",
            ELLIPSOID["WGS 84",6378137,298.257223563,
                LENGTHUNIT["metre",1]]],
        PRIMEM["Greenwich",0,
            ANGLEUNIT["degree",0.0174532925199433]],
        ID["EPSG",4326]],
    CONVERSION["UTM zone 42N",
        METHOD["Transverse Mercator",
            ID["EPSG",9807]],
        PARAMETER["Latitude of natural origin",0,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8801]],
        PARAMETER["Longitude of natural origin",69,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8802]],
        PARAMETER["Scale factor at natural origin",0.9996,
            SCALEUNIT["unity",1],
            ID["EPSG",8805]],
        PARAMETER["False easting",500000,
            LENGTHUNIT["m

In [17]:
# Open the GeoTIFF file
with rasterio.open(intermediate_tif) as dataset:

    # Access bounding box (bounds)
    bounds = dataset.bounds
    print(f"Bounds:    BoundingBox(left={bounds.left:.3f}, bottom={bounds.bottom:.3f}, right={bounds.right:.3f}, top={bounds.top:.3f})")
    
    # Access the CRS directly
    crs = dataset.crs
    
    # Print the EPSG code, if available
    if crs.is_epsg_code:
        print("EPSG Code:", crs.to_epsg())
    else:
        print("CRS is not an EPSG: ", crs.to_string())

Bounds:    BoundingBox(left=89841.237, bottom=2544694.968, right=908030.642, top=4213370.730)
EPSG Code: 32642


## 7 Resample Data

Resample the data in UTM using aoi_config.RESAMPLE which is measured in meters.

In [18]:
input_tif = intermediate_tif 

step = "_" + case + "_4_resampled_" + data_config.RESAMPLE_ALG + ".tif"

intermediate_tif = os.path.join(data_config.get_out_dir(), os.path.splitext(input_tif_filename)[0] + step)

print("input_tif:        ", input_tif)
print("\n")
print("intermediate_tif: ", intermediate_tif)
print("\n")


# Resample the data
gdal_resample(input_tif, 
              intermediate_tif, 
              data_config.RESAMPLE_ALG, 
              aoi_config.RESAMPLE, 
              aoi_config.RESAMPLE)

input_tif:         ./Rainfall/output/PK_42N/chirps-v2.0.2023_PK_42N_avg_PK_42N_3_utm.tif


intermediate_tif:  ./Rainfall/output/PK_42N/chirps-v2.0.2023_PK_42N_avg_PK_42N_4_resampled_average.tif


Creating output file that is 2045P x 4172L.
Processing ./Rainfall/output/PK_42N/chirps-v2.0.2023_PK_42N_avg_PK_42N_3_utm.tif [1/1] : 0Using internal nodata values (e.g. -999) for image ./Rainfall/output/PK_42N/chirps-v2.0.2023_PK_42N_avg_PK_42N_3_utm.tif.
Copying nodata values from source ./Rainfall/output/PK_42N/chirps-v2.0.2023_PK_42N_avg_PK_42N_3_utm.tif to destination ./Rainfall/output/PK_42N/chirps-v2.0.2023_PK_42N_avg_PK_42N_4_resampled_average.tif.
...10...20...30...40...50...60...70...80...90...100 - done.



In [19]:
# Open the GeoTIFF file
with rasterio.open(intermediate_tif) as dataset:

    # Access bounding box (bounds)
    bounds = dataset.bounds
    print(f"Bounds:    BoundingBox(left={bounds.left:.3f}, bottom={bounds.bottom:.3f}, right={bounds.right:.3f}, top={bounds.top:.3f})")

    # Access the CRS directly
    crs = dataset.crs

    # Print the EPSG code, if available
    if crs.is_epsg_code:
        print("EPSG Code:", crs.to_epsg())
    else:
        print("CRS is not an EPSG: ", crs.to_string())

Bounds:    BoundingBox(left=89841.237, bottom=2544570.730, right=907841.237, top=4213370.730)
EPSG Code: 32642


##  Information Summary: Resampled Data

In [20]:
if data_config.GDAL_INFO:
    run_gdalinfo(intermediate_tif)

Driver: GTiff/GeoTIFF
Files: ./Rainfall/output/PK_42N/chirps-v2.0.2023_PK_42N_avg_PK_42N_4_resampled_average.tif
Size is 2045, 4172
Coordinate System is:
PROJCRS["WGS 84 / UTM zone 42N",
    BASEGEOGCRS["WGS 84",
        DATUM["World Geodetic System 1984",
            ELLIPSOID["WGS 84",6378137,298.257223563,
                LENGTHUNIT["metre",1]]],
        PRIMEM["Greenwich",0,
            ANGLEUNIT["degree",0.0174532925199433]],
        ID["EPSG",4326]],
    CONVERSION["UTM zone 42N",
        METHOD["Transverse Mercator",
            ID["EPSG",9807]],
        PARAMETER["Latitude of natural origin",0,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8801]],
        PARAMETER["Longitude of natural origin",69,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8802]],
        PARAMETER["Scale factor at natural origin",0.9996,
            SCALEUNIT["unity",1],
            ID["EPSG",8805]],
        PARAMETER["False easting",500000,
         

In [21]:
# Open the GeoTIFF file
with rasterio.open(intermediate_tif) as dataset:

    # Access bounding box (bounds)
    bounds = dataset.bounds
    print(f"Bounds:    BoundingBox(left={bounds.left:.3f}, bottom={bounds.bottom:.3f}, right={bounds.right:.3f}, top={bounds.top:.3f})")

    # Print the EPSG code, if available
    if crs.is_epsg_code:
        print("EPSG Code:", crs.to_epsg())
    else:
        print("CRS is not an EPSG: ", crs.to_string())

Bounds:    BoundingBox(left=89841.237, bottom=2544570.730, right=907841.237, top=4213370.730)
EPSG Code: 32642
